# Install Packages and Setup Variables


In [ ]:
%pip install -q cohere==5.13.12

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

os.getenv("COHERE_API_KEY")
os. getenv("GOOGLE_API_KEY")


# Load Dataset


The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model.


In [ ]:
import urllib.request
import os
PROJECT_ROOT="<Root Directory path of your project>"


# url = 'https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv'
# filename =  os.path.join(PROJECT_ROOT,"mini-llama-articles.csv")

url = 'https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv'
filename =  os.path.join(PROJECT_ROOT,"mini-llama-articles-with_embeddings.csv")

urllib.request.urlretrieve(url, filename)

#On jupyter notebook use wget
#!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
#!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv

# Generate Embedding for data retrieved from a URL



To reference Cohere's embed  API refer to : https://docs.cohere.com/v2/reference/embed

To pick the "*input_type*" for "*embed*" API  refer to :https://docs.cohere.com/v2/docs/embeddings

Alternatively OpenAI embeddings can be referneced here: https://platform.openai.com/docs/guides/embeddings?lang=python

In [36]:
def split_into_chunks(text, chunk_size=1024):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i : i + chunk_size])

    return chunks

In [37]:
import csv
chunks = []

with open(filename, mode="r", encoding="utf-8") as file:
    csv_reader = csv.reader(file)
    
    for idx, row in enumerate(csv_reader):
        if idx == 0:
            continue
            # Skip header row
        chunks.extend(split_into_chunks(row[1]))

In [38]:
import pandas as pd

df = pd.DataFrame(chunks, columns=["chunk"])

df.keys()

Index(['chunk'], dtype='object')

Function to generate embeddings

In [40]:
def get_embedding(text):
    try:
        # Remove newlines
        text = text.replace("\n", " ")
        res = client.embed(
            texts=[text],
            model="embed-english-v3.0",
            input_type="search_document",
            embedding_types=["float"],
        )

        return res.embeddings.float

    except Exception as e:
        print(f"Error in get_embedding: {e}")  # Print the actual error message
        return None

Generate embeddigns for the data retrived from https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

In [43]:
generate_embeddings = False

In [ ]:
# Test the function
text_to_embed = "How many parameters LLaMA2 model has?"
#"Hello World"
embedding = get_embedding(text_to_embed)
print(embedding)

[[-0.051940918, 0.016220093, -0.026977539, 0.023254395, -0.03265381, 0.0013895035, -0.059295654, -0.034088135, -0.0793457, 0.018936157, 0.08276367, -0.009437561, -0.025558472, -0.076660156, 0.061950684, -0.005760193, 0.020645142, 0.029251099, -0.014053345, -0.015853882, -0.026535034, 0.029251099, -0.073913574, 0.097473145, 0.031585693, 0.010421753, -0.08892822, 0.040863037, 1.0788441e-05, 0.021209717, 0.008575439, -0.010910034, 0.020858765, 0.029556274, -0.022888184, 0.051086426, -0.018692017, 0.031188965, 0.023742676, -0.009811401, 0.04006958, -0.031433105, 0.04510498, 0.00121212, -0.00060653687, -0.019332886, 0.05722046, -0.0134887695, -0.040740967, -0.049560547, 0.00013530254, -0.0015735626, 0.018463135, -0.0020122528, 0.013435364, 0.008102417, 0.0151901245, -0.006816864, 0.019241333, -0.00642395, -0.03918457, 0.0211792, 0.032043457, -0.013908386, 0.015731812, -0.017593384, -0.043548584, 0.0345459, 0.025482178, 0.017425537, -0.0032138824, -0.03375244, 0.0129776, -0.03265381, 0.01543

In [47]:
#Flag to contorl the number of times  embeddings will be generated. We are limited by the number of API calls because this is a free account.
load_embedding_from_csv = True

In [90]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

# Generate embedding
if not load_embedding_from_csv:
    print("Generating embeddings...")
    embeddings = []
    for index, row in tqdm(df.iterrows()):
        # df.at[index, 'embedding'] = get_embedding( row['chunk'] )
        embeddings.append(get_embedding(row["chunk"]))

    embeddings_values = pd.Series(embeddings)
    df.insert(loc=1, column="embedding", value=embeddings_values)

# Or, load the embedding from the file.
else:
    print("Loaded the embedding file.")
    # Load the file as a CSV
    df = pd.read_csv(filename)
    # Convert embedding column to an array
    df["embedding"] = df["embedding"].apply(lambda x: np.array(eval(x)), 0)

Loaded the embedding file.


# User Question


In [91]:
# Define the user question, and convert it to embedding.
QUESTION = "How many parameters LLaMA2 model has?"
QUESTION_emb = get_embedding(QUESTION)

# Test Cosine Similarity


Calculating the similarity of embedding representations can help us to find pieces of text that are close to each other. In the following sample you see how the Cosine Similarity metric can identify which sentence could be a possible answer for the given user question. Obviously, the unrelated answer will score lower.


In [119]:
BAD_SOURCE_emb = get_embedding("The sky is blue.")
GOOD_SOURCE_emb = get_embedding("LLaMA2 model has a total of 2B parameters.")

In [120]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# A sample that how a good piece of text can achieve high similarity score compared
# to a completely unrelated text.
print("> Bad Response Score:", cosine_similarity(QUESTION_emb, BAD_SOURCE_emb))
print("> Good Response Score:", cosine_similarity(QUESTION_emb, GOOD_SOURCE_emb))

UTILITY FUNCTIONS

In [ ]:

import numpy as np

# Convert list to numpy array if it's not already
if isinstance(BAD_SOURCE_emb, list):
    BAD_SOURCE_emb = np.array(BAD_SOURCE_emb)

# Now you can check the shape
print(f"Shape of BAD_SOURCE_emb: {BAD_SOURCE_emb.shape}")
print(f"Type of BAD_SOURCE_emb: {type(BAD_SOURCE_emb)}")

Shape of BAD_SOURCE_emb: (1, 1024)
Type of BAD_SOURCE_emb: <class 'numpy.ndarray'>


The code below is added to resolve the error thrown by the cosine_similarity API due to the difference in dimensions of the objects to compare.

In [ ]:

import numpy as np
from sklearn.decomposition import PCA

# Since QUESTION_emb is 1024 dimensions and df['embedding'] is 1536
#Reduce QUESTION_emb to 174 dimensions
pca = PCA(n_components=174)
reduced_question_emb = pca.fit_transform(QUESTION_emb)

# # Then reduce df['embedding'] to the same 1024 dimensions
  #

# Create and fit PCA model
pca = PCA(n_components=174)  # Initialize PCA object
embeddings_array = np.stack([np.array(x) for x in df['embedding']])
reduced_df_emb = pca.fit_transform(embeddings_array)  # Fit and transform the data

In [ ]:
# The similarity between the questions and each part of the essay.
cosine_similarities = cosine_similarity(reduced_question_emb, reduced_df_emb)

print(cosine_similarities)

[[ 1.          0.56942815  0.00202648 ... -0.05718282  0.12496564
   0.16134031]
 [ 0.56942815  1.          0.03659397 ...  0.10547857  0.20367683
   0.14249395]
 [ 0.00202648  0.03659397  1.         ... -0.02722841 -0.12049013
  -0.118553  ]
 ...
 [-0.05718282  0.10547857 -0.02722841 ...  1.          0.3212876
   0.27095496]
 [ 0.12496564  0.20367683 -0.12049013 ...  0.3212876   1.
   0.47629768]
 [ 0.16134031  0.14249395 -0.118553   ...  0.27095496  0.47629768
   1.        ]]


In [ ]:
import numpy as np

number_of_chunks_to_retrieve = 3

# Sort the scores
highest_index = np.argmax(cosine_similarities)

# Pick the N highest scored chunks
indices = np.argsort(cosine_similarities[0])[::-1][:number_of_chunks_to_retrieve]
print(indices)

[ 0 95  1]


In [ ]:
# Look at the highest scored retrieved pieces of text
for idx, item in enumerate(df.chunk[indices]):
    print(f"> Chunk {idx+1}")
    print(item)
    print("----")

> Chunk 1
LLM Variants and Meta's Open Source Before shedding light on four major trends, I'd share the latest Meta's Llama 2 and Code Llama. Meta's Llama 2 represents a sophisticated evolution in LLMs. This suite spans models pretrained and fine-tuned across a parameter spectrum of 7 billion to 70 billion. A specialized derivative, Llama 2-Chat, has been engineered explicitly for dialogue-centric applications. Benchmarking revealed Llama 2's superior performance over most extant open-source chat models. Human-centric evaluations, focusing on safety and utility metrics, positioned Llama 2-Chat as a potential contender against proprietary, closed-source counterparts. The development trajectory of Llama 2 emphasized rigorous fine-tuning methodologies. Meta's transparent delineation of these processes aims to catalyze community-driven advancements in LLMs, underscoring a commitment to collaborative and responsible AI development. Code Llama is built on top of Llama 2 and is available in t

The chunks retrieved are bad.
[TODO]: Did the PCA mess up the chunk retrieval?

In [ ]:
%pip install google-generativeai==0.8.4

# Augment the Prompt


In [ ]:
import google.generativeai as genai
# Use the Gemini API to answer the questions based on the retrieved pieces of text.
try:
    # Formulating the system prompt and condition the model to answer only AI-related questions.
    system_prompt = (
        "You are an assistant and expert in answering questions from a chunks of content. "
        "Only answer AI-related question, else say that you cannot answer this question."
    )

    # Create a user prompt with the user's question
    prompt = (
        "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
        "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )
    # Add the retrieved pieces of text to the prompt.
    prompt = prompt.format("".join(df.chunk[indices]), QUESTION)

    model = genai.GenerativeModel(model_name= "gemini-1.5-flash", system_instruction=system_prompt)

    result = model.generate_content(prompt,request_options={"timeout": 1000},)
    res = result.text

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
print(res)

LLaMA 2 is available in four sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.



## How Augmenting the Prompt can address knowledge cutoff limitations and hallucinations

In [ ]:
# Consider this as a retrieved chunk
# https://ai.meta.com/blog/meta-llama-3-1/
Example_chunk = """
Introducing Llama 3.1 Llama 3.1 405B is the first openly available model that rivals the top AI models when it comes to state-of-the-art capabilities in general knowledge, steerability, math, tool use, and multilingual translation. With the release of the 405B model, we’re poised to supercharge innovation—with unprecedented opportunities for growth and exploration. We believe the latest generation of Llama will ignite new applications and modeling paradigms, including synthetic data generation to enable the improvement and training of smaller models, as well as model distillation—a capability that has never been achieved at this scale in open source.
As part of this latest release, we’re introducing upgraded versions of the 8B and 70B models. These are multilingual and have a significantly longer context length of 128K, state-of-the-art tool use, and overall stronger reasoning capabilities. This enables our latest models to support advanced use cases, such as long-form text summarization, multilingual conversational agents, and coding assistants. We’ve also made changes to our license, allowing developers to use the outputs from Llama models—including the 405B—to improve other models. True to our commitment to open source, starting today, we’re making these models available to the community for download on llama.meta.com and Hugging Face and available for immediate development on our broad ecosystem of partner platforms. Model evaluations
For this release, we evaluated performance on over 150 benchmark datasets that span a wide range of languages. In addition, we performed extensive human evaluations that compare Llama 3.1 with competing models in real-world scenarios. Our experimental evaluation suggests that our flagship model is competitive with leading foundation models across a range of tasks, including GPT-4, GPT-4o, and Claude 3.5 Sonnet. Additionally, our smaller models are competitive with closed and open models that have a similar number of parameters.
Model Architecture As our largest model yet, training Llama 3.1 405B on over 15 trillion tokens was a major challenge. To enable training runs at this scale and achieve the results we have in a reasonable amount of time, we significantly optimized our full training stack and pushed our model training to over 16 thousand H100 GPUs, making the 405B the first Llama model trained at this scale.
To address this, we made design choices that focus on keeping the model development process scalable and straightforward. We opted for a standard decoder-only transformer model architecture with minor adaptations rather than a mixture-of-experts model to maximize training stability.
We adopted an iterative post-training procedure, where each round uses supervised fine-tuning and direct preference optimization. This enabled us to create the highest quality synthetic data for each round and improve each capability’s performance.
Compared to previous versions of Llama, we improved both the quantity and quality of the data we use for pre- and post-training. These improvements include the development of more careful pre-processing and curation pipelines for pre-training data, the development of more rigorous quality assurance, and filtering approaches for post-training data.
"""

In [ ]:
QUESTION = "How many parameters LLaMA 3.1 model has?"

# Formulating the system prompt
system_prompt = (
        "You are an assistant and expert in answering questions from a chunks of content. "
        "Only answer AI-related question, else say that you cannot answer this question."
    )
# Combining the system prompt with the user's question
prompt = (
        "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
        "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )
prompt = prompt.format(Example_chunk, QUESTION)

model = genai.GenerativeModel(model_name= "gemini-1.5-flash", system_instruction=system_prompt)

#Gemini API call
result = model.generate_content(prompt,request_options={"timeout": 1000},)
res = result.text
print(res)

The LLaMA 3.1 model has **405 billion parameters**. 



# Without Augmentation


Test the Gemini API to answer the same question without the addition of retrieved documents. Basically, the LLM will use its knowledge to answer the question.


In [ ]:
QUESTION = "How many parameters LLaMA 3 model has?"

# Formulating the system prompt
system_prompt = "You are an assistant and expert in answering questions."

# Combining the system prompt with the user's question
prompt = "Be concise and take your time to answer the following question. \nQuestion: {}\nAnswer:"
prompt = prompt.format(QUESTION)

model = genai.GenerativeModel(model_name= "gemini-1.5-flash", system_instruction=system_prompt)

#Gemini API call
result = model.generate_content(prompt,request_options={"timeout": 1000},)
res = result.text

In [ ]:
print(res)

LLaMA 3 has **34 billion parameters**. 

